# Create CIGALE data file for (VIKING+DES+CATWISE) sources

Select a subsample of VIKING+DES+CATWISE sources that have the same distribution as EMU sources in a given magnitude.

# Import Libraries

In [1]:
import numpy as np
from astropy.table import Table
import pandas as pd
import matplotlib.pyplot as plt
import missingno

# Define the directory

In [2]:
observatory = '/home/mfonseca/fastdisk'
my_computer = '/home/polaris/Lab_Astro/data/'

directory = my_computer

# Import data

## Import EMU+VIKING+DES+CATWISE crossmatch data

In [3]:
emu_matched_no_mags_path = directory + 'cross_match/EMU_0102-32_DESY6GOLD_VIKING_CATWISE_noMags/EMU_0102-32_DESY6GOLD_VIKINGDR5_CATWISE_noMag_petro_2.fits'
emu_matched_no_mags = Table.read(emu_matched_no_mags_path).filled()
emu_matched_no_mags = emu_matched_no_mags.to_pandas()

match_mask = (emu_matched_no_mags['match_flag'] == 1) & (emu_matched_no_mags['p_i'] >= 0.1) & (emu_matched_no_mags['p_any'] >=0.83)
catalog_matches_noMags = emu_matched_no_mags[match_mask]
print(f'Number of objects in the catalog {len(catalog_matches_noMags)}')

# Here we select objects that have matches in all three surveys
full_match_mask = (catalog_matches_noMags['DESY6_coadd_object_id'] != -99) & (catalog_matches_noMags['VKG_sourceID'] != -99) & (catalog_matches_noMags['CAT_source_id'] != b'-99')
emu_catalog_matches_noMags_full = catalog_matches_noMags[full_match_mask]

# Ignore the warning
emu_catalog_matches_noMags_full['EMU_island_name'] = emu_catalog_matches_noMags_full['EMU_island_name'].str.decode('utf-8').str.strip(' ')

print(f'Number of objects in the catalog that have matches in all three surveys {len(emu_catalog_matches_noMags_full)}')

emu_catalog_matches_noMags_full.drop(columns=['DESY6_mag_auto_y_extcorr', 'DESY6_z_y_extcorr','VKG_magerr_petro_z', 'VKG_mag_petro_z_ab_extcorr'], inplace=True)
emu_catalog_matches_noMags_full.dropna(inplace=True)

print(f'Number of objects in the catalog that have matches in all three surveys and have full photometry {len(emu_catalog_matches_noMags_full)}')

Number of objects in the catalog 11695
Number of objects in the catalog that have matches in all three surveys 3855
Number of objects in the catalog that have matches in all three surveys and have full photometry 3530


/tmp/ipykernel_73934/2011678717.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  emu_catalog_matches_noMags_full['EMU_island_name'] = emu_catalog_matches_noMags_full['EMU_island_name'].str.decode('utf-8').str.strip(' ')
/tmp/ipykernel_73934/2011678717.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  emu_catalog_matches_noMags_full.drop(columns=['DESY6_mag_auto_y_extcorr', 'DESY6_z_y_extcorr','VKG_magerr_petro_z', 'VKG_mag_petro_z_ab_extcorr'], inplace=True)
/tmp/ipykernel_73934/2011678717.py:19: SettingWithCopyWarning: 
A value is trying to be set on a co

## Import VIKING+DES+CATWISE crossmatch data

In [5]:
other_matched_no_mags_path = directory + 'cross_match/without_EMU/DESY6_VKG_CAT_noMags/DESY6_VKG_CAT_noMags.fits'
other_matched_no_mags = Table.read(other_matched_no_mags_path).filled()
other_matched_no_mags = other_matched_no_mags.to_pandas()

# Match criteria from NWAY
match_mask = (other_matched_no_mags['match_flag'] == 1) & (other_matched_no_mags['p_i'] >= 0.1) & (other_matched_no_mags['p_any'] >=0.00)
other_catalog_matches_noMags = other_matched_no_mags[match_mask]
print(f'Number of objects in the catalog {len(other_catalog_matches_noMags)}')

# Here we select objects that have matches in all three surveys
full_match_mask = (other_catalog_matches_noMags['DESY6_coadd_object_id'] != -99) & (other_catalog_matches_noMags['VKG_sourceID'] != -99) & (other_catalog_matches_noMags['CAT_source_id'] != b'-99')
other_catalog_matches_noMags_full = other_catalog_matches_noMags[full_match_mask]

print(f'Number of objects in the catalog that have matches in all three surveys {len(other_catalog_matches_noMags_full)}')

# Remove objects with incomplete photometry
other_catalog_matches_noMags_full.dropna(inplace=True)

print(f'Number of objects in the catalog that have matches in all three surveys and have full photometry {len(other_catalog_matches_noMags_full)}')

Number of objects in the catalog 591080
Number of objects in the catalog that have matches in all three surveys 366961
Number of objects in the catalog that have matches in all three surveys and have full photometry 315399


/tmp/ipykernel_73934/3434645965.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  other_catalog_matches_noMags_full.dropna(inplace=True)


## Remove EMU+VIKING+DES+CATWISE crossmatch data from VIKING+DES 

In [6]:
# List of id columns
cols = ['DESY6_coadd_object_id', 'VKG_sourceID', 'CAT_source_id']

# Create a boolean mask for each column: True where df1[col] is in df2[col]
mask = pd.Series(False, index=other_catalog_matches_noMags_full.index)  # initialize False mask

for col in cols:
    mask |= other_catalog_matches_noMags_full[col].isin(emu_catalog_matches_noMags_full[col])

# Select only rows in df1 where mask is False (i.e., no match on any column)
other_catalog_matches_noMags_full_filtered = other_catalog_matches_noMags_full[~mask].copy()

print(f'Number of objects in the catalog that have matches in all three surveys and have full photometry but are not in EMU {len(other_catalog_matches_noMags_full_filtered)}')

Number of objects in the catalog that have matches in all three surveys and have full photometry but are not in EMU 311164


# Select a subsample of VIKING+DES+CATWISE with similar distribution to EMU+VIKING+DES+CATWISE

# Save the subsample to a CIGALE file